In [1]:
%pip install streamlit
%pip install matplotlib
%pip install seaborn
%pip install scikit-learn
%pip install plotly
%pip install statsmodels

#Para instalar npm en visual studio
#1.Desde Google escribir node.js
#2. Instalar la versi√≥n m√°s recomendada
! npm install localtunnel


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip3.12 in

In [26]:
%%writefile EVIDENCIA_2.py

## streamlit run EVIDENCIA_2.py

import os
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import streamlit as st

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import (
    r2_score, mean_absolute_error, mean_squared_error,
    precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix, roc_curve
)
from scipy.optimize import curve_fit

# =========================
# Utilidades base
# =========================
def safe_read_csv(path):
    try:
        if os.path.exists(path):
            return pd.read_csv(path)
    except Exception:
        pass
    return None

def unify_schema(df, city_name=None, verbose=False):
    """
    Normaliza nombres, resuelve columnas duplicadas, agrega 'city',
    y convierte strings con '%' a num√©ricos. Evita AttributeError.
    """
    if df is None or len(df) == 0:
        return None

    # 1) Normaliza nombres a snake_case
    cols_raw = list(df.columns)
    norm_cols = [str(c).strip().lower().replace(" ", "_") for c in cols_raw]

    # 2) Unifica duplicados: col, col__1, col__2, ...
    seen = {}
    unique_cols = []
    dups = []
    for c in norm_cols:
        if c not in seen:
            seen[c] = 0
            unique_cols.append(c)
        else:
            seen[c] += 1
            new_c = f"{c}__{seen[c]}"
            unique_cols.append(new_c)
            dups.append((c, new_c))
    if verbose and dups:
        print("Columnas duplicadas renombradas:", dups)

    df = df.copy()
    df.columns = unique_cols

    # 3) Correcciones habituales
    ren = {
        "accomodates": "accommodates",
        "bedroms": "bedrooms",
        "bathroom": "bathrooms",
        "host_response_time(%)": "host_response_rate",
        "review_scores_value": "review_scores_rating"
    }
    for k, v in ren.items():
        if k in df.columns and v not in df.columns:
            df.rename(columns={k: v}, inplace=True)

    # 4) Asegura 'city'
    if "city" not in df.columns:
        df["city"] = city_name if city_name else "unknown"
    else:
        if city_name:
            df["city"] = df["city"].fillna(city_name).replace("", city_name)

    # 5) Convierte columnas con s√≠mbolo %
    for c in df.columns:
        s = df[c]
        if pd.api.types.is_object_dtype(s):
            if s.astype(str).str.contains("%").any():
                s2 = s.astype(str).str.replace("%", "", regex=False)
                s2 = pd.to_numeric(s2, errors="coerce")
                df[c] = s2

    return df

def numeric_cols(df):
    return [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]

def categorical_cols(df):
    return [c for c in df.columns if (df[c].dtype == "object" or str(df[c].dtype).startswith("category"))]

def r_from_y_ypred(y_true, y_pred):
    if len(y_true) < 2 or len(y_pred) < 2:
        return np.nan
    try:
        return float(np.corrcoef(y_true, y_pred)[0,1])
    except Exception:
        return np.nan

def fit_nonlinear_series(x, y, model_name):
    def f_quad(z, a, b, c): return a*z**2 + b*z + c
    def f_exp(z, a, b, c):  return a*np.exp(b*z) + c
    def f_inv(z, a, b):     return a + b/np.where(z==0, np.finfo(float).eps, z)

    if model_name == "Cuadr√°tica":
        popt, _ = curve_fit(f_quad, x, y, maxfev=20000)
        yhat = f_quad(x, *popt)
        return yhat, popt
    elif model_name == "Exponencial":
        p0 = (1.0, 0.01, float(np.nanmedian(y)))
        popt, _ = curve_fit(f_exp, x, y, p0=p0, maxfev=20000)
        yhat = f_exp(x, *popt)
        return yhat, popt
    else:  # Inversa
        popt, _ = curve_fit(f_inv, x, y, maxfev=20000)
        yhat = f_inv(x, *popt)
        return yhat, popt

def coerce_numeric_cols(df, cols):
    """Devuelve una copia donde 'cols' se convierten a num√©rico con errors='coerce'."""
    d = df.copy()
    for c in cols:
        if c in d.columns:
            d[c] = pd.to_numeric(d[c], errors='coerce')
    return d

# =========================
# Configuraci√≥n app
# =========================
st.set_page_config(
    page_title="Dashboard Mult-ciudad Airbnb: Proyecto Innovativo Grupo IPYNB",
    page_icon="üè†",
    layout="wide"
)

# --- Portada (Imagen + T√≠tulo + Descripci√≥n) ---
# Ruta proporcionada por el usuario (c√°mbiala si tu archivo est√° en otra ubicaci√≥n)
PORTADA_PATHS = [
    "WhatsApp Image 2025-10-22 at 11.38.03 PM.jpeg",
    "./assets/aura.jpg",
    "./assets/aura.png",
    "/mnt/data/WhatsApp Image 2025-10-22 at 11.38.03 PM.jpeg",
]
img_path = next((p for p in PORTADA_PATHS if os.path.exists(p)), None)
if img_path:
    col_l, col_c, col_r = st.columns([1,2,1])
    with col_c:
        st.image(img_path, use_column_width=True)
else:
    st.info("‚ö†Ô∏è Imagen de portada no encontrada. Verifica la ruta en PORTADA_PATHS.")

st.title("Dashboard Mult-ciudad Airbnb: Proyecto Innovativo Grupo IPYNB")
st.write("**Este dashboard permite establecer un panorama completo de Airbnb, sus Hosts y Alojamientos, potencializando al Programa AURA.**")

# =========================
# Sidebar
# =========================
with st.sidebar:
    st.header("‚öôÔ∏è Configuraci√≥n")
    color = st.color_picker("Color principal", "#0B6E4F")
    grid_alpha = st.slider("Transparencia de rejilla", 0.0, 1.0, 0.2, 0.05)
    st.markdown("---")
    st.subheader("üìÇ Carga de datos (opcional)")
    ups = st.file_uploader("Sube CSV (puedes m√∫ltiples)", type=["csv"], accept_multiple_files=True)

# =========================
# Carga de datasets base
# =========================
df_mex = unify_schema(safe_read_csv("Mexico_Limpio.csv"), "Mexico")
df_mad = unify_schema(safe_read_csv("Madrid_AirBnb_010.csv"), "Madrid")
df_val = unify_schema(safe_read_csv("VALENCIA_LIMPIO.csv"), "Valencia")
df_rio = unify_schema(safe_read_csv("base_final.csv"), "Rio")   # <- etiqueta "Rio"
df_roma = unify_schema(safe_read_csv("df_limp_ROMA.csv"), "Roma")

dfs_raw = [d for d in [df_mex, df_mad, df_val, df_rio, df_roma] if d is not None]

if ups:
    for f in ups:
        try:
            tmp = pd.read_csv(f)
            dfs_raw.append(unify_schema(tmp, city_name=os.path.splitext(f.name)[0]))
        except Exception as e:
            st.warning(f"No se pudo leer {f.name}: {e}")

if not dfs_raw:
    st.error("No se encontraron datos. Coloca los CSV en la carpeta o s√∫belos.")
    st.stop()

# Intersecci√≥n de columnas comunes para asegurar comparabilidad
common_cols = set(dfs_raw[0].columns)
for d in dfs_raw[1:]:
    common_cols = common_cols.intersection(set(d.columns))
common_cols = sorted(list(common_cols))

# Reindexa cada DF a columnas comunes
dfs = [d[common_cols].copy() for d in dfs_raw]

# Concat para an√°lisis global
df_all = pd.concat(dfs, ignore_index=True)
cities_all = sorted(df_all["city"].dropna().astype(str).unique().tolist())

# Selecci√≥n de ciudades a comparar
sel_cities = st.multiselect("üåç Ciudades a comparar", options=cities_all, default=cities_all)
if not sel_cities:
    st.warning("Selecciona al menos una ciudad.")
    st.stop()
df = df_all[df_all["city"].isin(sel_cities)].reset_index(drop=True)

# Columnas por tipo (en universo com√∫n)
num_cols_all = [c for c in common_cols if pd.api.types.is_numeric_dtype(df[c])]
cat_cols_all = [c for c in common_cols if (df[c].dtype == "object" or str(df[c].dtype).startswith("category"))]

# =========================
# Tabs (7 secciones)
# =========================
tabs = st.tabs([
    "1) üß∞ Extracci√≥n",
    "2) üî† Categ√≥ricas",
    "3) üìà Regresi√≥n Lineal Simple",
    "4) üßÆ Regresi√≥n Lineal M√∫ltiple",
    "5) üåÄ Regresi√≥n No Lineal",
    "6) üß™ Regresi√≥n Log√≠stica",
    "7) üí° Insights & Propuestas",
])

# ============================================================
# TAB 1 - Extracci√≥n (comparativos de variables de servicio)
# ============================================================
with tabs[0]:
    st.subheader("üß∞ Extracci√≥n: comparativo de variables clave por ciudad")
    st.caption("Variables t√≠picas: price, number_of_reviews, availability_365, calculated_host_listings_count, accommodates, reviews_per_month, minimum_nights, estimated_occupancy_l365d, review_scores_rating.")

    candidates = [
        "price", "number_of_reviews", "availability_365",
        "calculated_host_listings_count", "host_total_listings_count",
        "accommodates", "reviews_per_month", "minimum_nights",
        "estimated_occupancy_l365d", "review_scores_rating"
    ]
    present = [c for c in candidates if c in common_cols]
    sel_vars = st.multiselect("Variables a comparar", options=present, default=present)

    if not sel_vars:
        st.info("Selecciona al menos una variable.")
    else:
        # Conversi√≥n segura a num√©rico
        df_num = coerce_numeric_cols(df, sel_vars)

        # KPIs por ciudad (mediana y promedio)
        try:
            kpi = (df_num.groupby("city")[sel_vars]
                   .agg(['median', 'mean'])
                   .round(3))
            st.dataframe(kpi, use_container_width=True)
        except Exception as e:
            st.warning(f"No fue posible calcular median/mean: {e}")

        # Gr√°ficas por variable (box + barras) con columnas num√©ricas
        for v in sel_vars:
            st.markdown(f"**Variable:** `{v}`")
            c1, c2 = st.columns(2)

            with c1:
                try:
                    fig = px.box(df_num, x="city", y=v, points="outliers", color="city",
                                 title=f"Distribuci√≥n de {v} por ciudad")
                    st.plotly_chart(fig, use_container_width=True)
                except Exception:
                    st.info(f"No se pudo graficar boxplot para {v}.")

            with c2:
                try:
                    # SeriesGroupBy.median sin numeric_only (ya es num√©rico)
                    agg = df_num.groupby("city")[v].median().reset_index()
                    fig2 = px.bar(agg, x="city", y=v, title=f"Mediana de {v} por ciudad")
                    st.plotly_chart(fig2, use_container_width=True)
                except Exception:
                    st.info(f"No se pudo graficar barras para {v}.")

# ============================================================
# TAB 2 - Categ√≥ricas (10 comunes)
# ============================================================
with tabs[1]:
    st.subheader("üî† An√°lisis de variables categ√≥ricas (reglas espec√≠ficas)")
    st.caption("room_type y host_is_superhost en todas; neighbourhood y host_response_time solo donde existan (no en M√©xico).")

    # --- Variables can√≥nicas a usar ---
    canonical = ["room_type", "host_is_superhost", "neighbourhood", "host_response_time"]

    # Aliases para ubicar columnas equivalentes por nombre
    alias_map = {
        "room_type": ["room_type", "room", "tipo_habitacion"],
        "host_is_superhost": ["host_is_superhost", "is_superhost", "superhost"],
        "neighbourhood": ["neighbourhood", "neighborhood", "neighbourhood_cleansed", "barrio", "district", "zona"],
        "host_response_time": ["host_response_time", "response_time", "tiempo_respuesta"],
    }

    def find_col(df_local, aliases):
        cols = list(df_local.columns)
        low_cols = [c.lower() for c in cols]
        # Exacta (case-insensitive)
        for a in aliases:
            if a.lower() in low_cols:
                return cols[low_cols.index(a.lower())]
        # Inclusi√≥n parcial
        for a in aliases:
            for i, c in enumerate(low_cols):
                if a.lower() in c:
                    return cols[i]
        return None

    def to_yes_no(series):
        s = series.astype(str).str.lower().str.strip()
        yes = {"1","t","true","yes","si","s√≠","y","s"}
        no  = {"0","f","false","no","n"}
        return np.where(s.isin(yes), "S√≠",
                        np.where(s.isin(no), "No", series.astype(str)))

    # --- Construcci√≥n del DataFrame categ√≥rico unificado ---
    cat_frames = []
    mapping_rows = []

    for d in dfs_raw:
        if d is None or "city" not in d.columns or d.empty:
            continue

        tmp = pd.DataFrame()
        tmp["city"] = d["city"].astype(str)

        # Detecta si el DF es de M√©xico (para excluir neighbourhood/response_time)
        df_has_mexico = tmp["city"].astype(str).str.lower().str.contains("mexico").any()

        # 1. room_type ‚Üí siempre
        src_room = find_col(d, alias_map["room_type"])
        tmp["room_type"] = d[src_room] if src_room else np.nan

        # 2. host_is_superhost ‚Üí siempre (convertido a S√≠/No)
        src_super = find_col(d, alias_map["host_is_superhost"])
        if src_super:
            tmp["host_is_superhost"] = pd.Series(to_yes_no(d[src_super])).astype("category")
        else:
            tmp["host_is_superhost"] = np.nan

        # 3. neighbourhood ‚Üí solo donde exista y no sea M√©xico
        src_neigh = find_col(d, alias_map["neighbourhood"])
        if (not df_has_mexico) and src_neigh:
            tmp["neighbourhood"] = d[src_neigh]
        else:
            tmp["neighbourhood"] = np.nan

        # 4. host_response_time ‚Üí solo donde exista y no sea M√©xico
        src_resp = find_col(d, alias_map["host_response_time"])
        if (not df_has_mexico) and src_resp:
            tmp["host_response_time"] = d[src_resp]
        else:
            tmp["host_response_time"] = np.nan

        cat_frames.append(tmp)

        # Para mostrar en la tabla qu√© columnas se usaron
        cities_str = ", ".join(sorted(tmp["city"].astype(str).unique().tolist()))
        mapping_rows.append({
            "Ciudad(es) en DF": cities_str,
            "room_type <-": src_room if src_room else "‚ùå",
            "host_is_superhost <-": src_super if src_super else "‚ùå",
            "neighbourhood <-": (src_neigh if (src_neigh and not df_has_mexico) else "‚ùå (descartado o no existe)"),
            "host_response_time <-": (src_resp if (src_resp and not df_has_mexico) else "‚ùå (descartado o no existe)"),
        })

    if not cat_frames:
        st.warning("No se pudieron construir variables categ√≥ricas desde los datasets originales.")
        st.stop()

    df_cat_all = pd.concat(cat_frames, ignore_index=True)
    df_cat = df_cat_all[df_cat_all["city"].isin(sel_cities)].copy()

    if df_cat.empty:
        st.warning("No hay registros categ√≥ricos en las ciudades seleccionadas.")
        st.stop()

    # Mostrar mapeo de columnas detectadas por dataset/ciudad
    with st.expander("Ver mapeo de columnas utilizadas por dataset/ciudad"):
        st.dataframe(pd.DataFrame(mapping_rows), use_container_width=True)

    # Filtra solo las variables realmente disponibles con datos
    present_cats = [c for c in canonical if c in df_cat.columns and df_cat[c].notna().any()]

    # Selector para activar/desactivar variables
    sel_cats = st.multiselect(
        "Selecciona variables a visualizar",
        options=present_cats,
        default=present_cats,
        max_selections=len(present_cats)
    )

    if not sel_cats:
        st.info("Selecciona al menos una variable categ√≥rica.")
    else:
        for cat in sel_cats:
            st.markdown(f"### üìä Variable: `{cat}`")

            # --- Distribuci√≥n global ---
            try:
                dist = df_cat[cat].astype("category").value_counts(dropna=False).reset_index()
                dist.columns = [cat, "Frecuencia"]

                col1, col2 = st.columns(2)
                with col1:
                    fig_bar = px.bar(
                        dist, x=cat, y="Frecuencia", color=cat,
                        title=f"Frecuencia global de {cat}"
                    )
                    st.plotly_chart(fig_bar, use_container_width=True)

                with col2:
                    fig_pie = px.pie(
                        dist, names=cat, values="Frecuencia",
                        title=f"Participaci√≥n global de {cat}"
                    )
                    st.plotly_chart(fig_pie, use_container_width=True)

            except Exception as e:
                st.warning(f"No se pudo graficar {cat}: {e}")

            # --- Comparativo por ciudad ---
            try:
                cross = (
                    df_cat.groupby(["city", cat])
                          .size()
                          .reset_index(name="Conteo")
                )
                fig_cross = px.bar(
                    cross, x="city", y="Conteo", color=cat,
                    barmode="group", title=f"{cat} por ciudad"
                )
                st.plotly_chart(fig_cross, use_container_width=True)
            except Exception as e:
                st.warning(f"No se pudo generar el comparativo de {cat} por ciudad: {e}")

        st.success("‚úÖ Tab 2 configurado seg√∫n reglas: room_type y superhost en todas; neighbourhood y response_time solo donde existan (no M√©xico).")

# ============================================================
# TAB 3 - Regresi√≥n Lineal Simple (por ciudad)
# ============================================================
with tabs[2]:
    st.subheader("üìà Regresi√≥n Lineal Simple (comparativa por ciudad)")
    if len(num_cols_all) < 2:
        st.info("Se requieren al menos dos variables num√©ricas comunes.")
    else:
        col1, col2 = st.columns(2)
        with col1:
            y_lin = st.selectbox("Y (num√©rica)", options=num_cols_all, key="lin_y")
        with col2:
            x_lin = st.selectbox("X (num√©rica)", options=[c for c in num_cols_all if c != y_lin], key="lin_x")

        sel_c_lin = st.multiselect("Ciudades a incluir", options=sel_cities, default=sel_cities, key="lin_cities")
        results = []
        for city in sel_c_lin:
            dfx = df[df["city"]==city][[x_lin, y_lin]].copy()
            dfx[x_lin] = pd.to_numeric(dfx[x_lin], errors='coerce')
            dfx[y_lin] = pd.to_numeric(dfx[y_lin], errors='coerce')
            dfx = dfx.dropna()
            if len(dfx) < 8:
                results.append({"city": city, "R2": np.nan, "R": np.nan, "n": len(dfx)})
                st.info(f"{city}: datos insuficientes.")
                continue
            X = dfx[[x_lin]].values
            Y = dfx[y_lin].values
            Xtr, Xte, Ytr, Yte = train_test_split(X, Y, test_size=0.3, random_state=42)
            model = LinearRegression().fit(Xtr, Ytr)
            yhat = model.predict(Xte)
            R2 = r2_score(Yte, yhat)
            R = r_from_y_ypred(Yte, yhat)
            results.append({"city": city, "R2": R2, "R": R, "n": len(dfx)})

            # Gr√°fico por ciudad
            fig, ax = plt.subplots()
            ax.scatter(X, Y, alpha=0.5)
            X_sorted = np.sort(X, axis=0)
            ax.plot(X_sorted, model.predict(X_sorted), color=color, lw=2)
            ax.set_title(f"{city} | {y_lin} ~ {x_lin}")
            ax.set_xlabel(x_lin); ax.set_ylabel(y_lin); ax.grid(alpha=grid_alpha)
            st.pyplot(fig, use_container_width=True)

        st.markdown("**Resumen de m√©tricas (test):**")
        st.dataframe(pd.DataFrame(results).round(3), use_container_width=True)

# ============================================================
# TAB 4 - Regresi√≥n Lineal M√∫ltiple (por ciudad)
# ============================================================
with tabs[3]:
    st.subheader("üßÆ Regresi√≥n Lineal M√∫ltiple (comparativa por ciudad)")
    if len(num_cols_all) < 3:
        st.info("Se requieren ‚â• 3 variables num√©ricas comunes para m√∫ltiple.")
    else:
        col1, col2 = st.columns([1,2])
        with col1:
            y_mul = st.selectbox("Y (num√©rica)", options=num_cols_all, key="mul_y")
        with col2:
            x_mul = st.multiselect("X (num√©ricas)", options=[c for c in num_cols_all if c != y_mul],
                                   default=[c for c in num_cols_all if c != y_mul][:3], key="mul_x")
        if not x_mul:
            st.info("Selecciona al menos una X.")
        else:
            sel_c_mul = st.multiselect("Ciudades a incluir", options=sel_cities, default=sel_cities, key="mul_cities")
            rows = []
            for city in sel_c_mul:
                cols = [y_mul] + x_mul
                dfx = df[df["city"]==city][cols].copy()
                for c in cols:
                    dfx[c] = pd.to_numeric(dfx[c], errors='coerce')
                dfx = dfx.dropna()
                if len(dfx) < 10:
                    rows.append({"city": city, "R2": np.nan, "R": np.nan, "n": len(dfx)})
                    st.info(f"{city}: datos insuficientes.")
                    continue
                X = dfx[x_mul].values
                Y = dfx[y_mul].values
                Xtr, Xte, Ytr, Yte = train_test_split(X, Y, test_size=0.3, random_state=42)
                model = LinearRegression().fit(Xtr, Ytr)
                yhat = model.predict(Xte)
                R2 = r2_score(Yte, yhat)
                R = r_from_y_ypred(Yte, yhat)
                rows.append({"city": city, "R2": R2, "R": R, "n": len(dfx)})

                # Gr√°fico real vs predicho
                fig, ax = plt.subplots()
                ax.scatter(Yte, yhat, alpha=0.6)
                m = [min(Yte.min(), yhat.min()), max(Yte.max(), yhat.max())]
                ax.plot(m, m, '--', lw=1)
                ax.set_title(f"{city} | {y_mul} ~ {', '.join(x_mul)}")
                ax.set_xlabel("Real"); ax.set_ylabel("Predicho"); ax.grid(alpha=grid_alpha)
                st.pyplot(fig, use_container_width=True)

            st.markdown("**Resumen de m√©tricas (test):**")
            st.dataframe(pd.DataFrame(rows).round(3), use_container_width=True)

# ============================================================
# TAB 5 - Regresi√≥n No Lineal (por ciudad)
# ============================================================
with tabs[4]:
    st.subheader("üåÄ Regresi√≥n No Lineal (comparativa por ciudad)")
    if len(num_cols_all) < 2:
        st.info("Se requieren al menos dos variables num√©ricas comunes.")
    else:
        c1, c2, c3 = st.columns(3)
        with c1:
            y_nl = st.selectbox("Y", options=num_cols_all, key="nl_y")
        with c2:
            x_nl = st.selectbox("X", options=[c for c in num_cols_all if c != y_nl], key="nl_x")
        with c3:
            model_nl = st.selectbox("Modelo", ["Cuadr√°tica", "Exponencial", "Inversa"], index=0)

        sel_c_nl = st.multiselect("Ciudades a incluir", options=sel_cities, default=sel_cities, key="nl_cities")
        rows = []
        for city in sel_c_nl:
            dfx = df[df["city"]==city][[x_nl, y_nl]].copy()
            dfx[x_nl] = pd.to_numeric(dfx[x_nl], errors='coerce')
            dfx[y_nl] = pd.to_numeric(dfx[y_nl], errors='coerce')
            dfx = dfx.dropna().sort_values(x_nl)
            if len(dfx) < 10:
                rows.append({"city": city, "R2": np.nan, "R": np.nan, "n": len(dfx)})
                st.info(f"{city}: datos insuficientes.")
                continue
            x = dfx[x_nl].astype(float).values
            y = dfx[y_nl].astype(float).values
            try:
                yhat, popt = fit_nonlinear_series(x, y, model_nl)
                R2 = r2_score(y, yhat)
                R = r_from_y_ypred(y, yhat)
                rows.append({"city": city, "R2": R2, "R": R, "n": len(dfx)})

                # Gr√°fico
                fig, ax = plt.subplots()
                ax.scatter(x, y, alpha=0.5)
                ax.plot(x, yhat, color=color, lw=2)
                ax.set_title(f"{city} | {y_nl} ~ {x_nl} ({model_nl})")
                ax.set_xlabel(x_nl); ax.set_ylabel(y_nl); ax.grid(alpha=grid_alpha)
                st.pyplot(fig, use_container_width=True)
            except Exception as e:
                rows.append({"city": city, "R2": np.nan, "R": np.nan, "n": len(dfx)})
                st.warning(f"{city}: fallo del ajuste ({e}).")

        st.markdown("**Resumen de m√©tricas (ajuste in-sample):**")
        st.dataframe(pd.DataFrame(rows).round(3), use_container_width=True)

# ============================================================
# TAB 6 - Regresi√≥n Log√≠stica (por ciudad)
# ============================================================
with tabs[5]:
    st.subheader("üß™ Regresi√≥n Log√≠stica (comparativa por ciudad)")
    cols_all = list(common_cols)
    y_choice = st.selectbox("Variable dependiente (Y)", options=cols_all, key="log_y")
    x_choices = st.multiselect("Variables explicativas (X)", options=[c for c in cols_all if c != y_choice], max_selections=6, key="log_xs")

    if not x_choices:
        st.info("Selecciona al menos una X.")
    else:
        dfY = df[y_choice]
        # --- Binarizaci√≥n Y (SOLO Media o Manual) ---
        st.markdown("**Binarizaci√≥n de Y**")
        if pd.api.types.is_numeric_dtype(dfY):
            howY = st.selectbox("M√©todo (Y num√©rica)", ["Media", "Manual"], index=0, key="y_thr_m")
            if howY == "Media":
                thrY = float(dfY.dropna().mean())
            else:
                thrY = st.number_input("Umbral manual Y", value=float(dfY.dropna().mean()), key="y_thr_i")
            y_bin = (dfY >= thrY).astype(int)
            st.caption(f"Y binaria: 1 si {y_choice} ‚â• {thrY:.4f}")
        else:
            uniq = dfY.dropna().astype(str).unique().tolist()
            pos = st.selectbox("Clase positiva (Y=1)", options=sorted(uniq), key="y_pos")
            y_bin = (dfY.astype(str) == pos).astype(int)
            st.caption(f"Y binaria: 1 si {y_choice} == '{pos}'")

        # --- Binarizaci√≥n opcional de UNA X num√©rica (SOLO Media o Manual) ---
        st.markdown("**Binarizaci√≥n opcional de X (una X num√©rica espec√≠fica)**")
        binarize_x = st.checkbox("Binarizar una X num√©rica", value=False, key="bx_flag")
        X_design = df[x_choices].copy()
        if binarize_x:
            x_num_candidates = [c for c in x_choices if pd.api.types.is_numeric_dtype(df[c])]
            if x_num_candidates:
                x_bin_name = st.selectbox("X a binarizar", options=x_num_candidates, key="bx_sel")
                howX = st.selectbox("M√©todo (X num√©rica)", ["Media", "Manual"], index=0, key="x_thr_m")
                if howX == "Media":
                    x_bin_thr = float(df[x_bin_name].dropna().mean())
                else:
                    x_bin_thr = st.number_input("Umbral manual X", value=float(df[x_bin_name].dropna().mean()), key="x_thr_i")
                X_design[x_bin_name] = (df[x_bin_name] >= x_bin_thr).astype(int)
                st.caption(f"{x_bin_name} binaria: 1 si ‚â• {x_bin_thr:.4f}")
            else:
                st.info("No hay X num√©ricas seleccionadas para binarizar.")

        # --- class_weight & split ---
        use_bal = st.checkbox("Usar class_weight='balanced'", value=True, key="cw")
        test_size = st.slider("Proporci√≥n de test", 0.1, 0.5, 0.3, 0.05, key="tsz")

        sel_c_log = st.multiselect("Ciudades a incluir", options=sel_cities, default=sel_cities, key="log_cities")
        rows = []
        for city in sel_c_log:
            dfx = X_design[df["city"]==city].copy()
            yy = y_bin[df["city"]==city].copy()
            # Conversi√≥n num√©rica de X por seguridad
            for c in dfx.columns:
                dfx[c] = pd.to_numeric(dfx[c], errors='coerce')
            data = pd.concat([dfx, yy.rename("__y")], axis=1).dropna()
            if data["__y"].nunique() < 2 or len(data) < 30:
                rows.append({"city": city, "AUC": np.nan, "Precision": np.nan, "Recall": np.nan, "F1": np.nan, "n": len(data)})
                st.info(f"{city}: datos insuficientes o una sola clase.")
                continue

            X = data.drop(columns="__y").values
            y = data["__y"].astype(int).values

            Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=test_size, random_state=42, stratify=y)
            pipe = Pipeline([
                ("scaler", StandardScaler(with_mean=True, with_std=True)),
                ("logit", LogisticRegression(max_iter=600, class_weight=("balanced" if use_bal else None)))
            ])
            pipe.fit(Xtr, ytr)
            proba = pipe.predict_proba(Xte)[:,1]
            pred = (proba >= 0.5).astype(int)

            AUC = roc_auc_score(yte, proba)
            PREC = precision_score(yte, pred, zero_division=0)
            REC = recall_score(yte, pred, zero_division=0)
            F1 = f1_score(yte, pred, zero_division=0)
            rows.append({"city": city, "AUC": AUC, "Precision": PREC, "Recall": REC, "F1": F1, "n": len(data)})

            # Gr√°fico ROC por ciudad
            fpr, tpr, thr = roc_curve(yte, proba)
            fig, ax = plt.subplots()
            ax.plot(fpr, tpr, lw=2, label=f"AUC={AUC:.3f}")
            ax.plot([0,1],[0,1],'--', lw=1)
            ax.set_title(f"ROC - {city}")
            ax.set_xlabel("FPR"); ax.set_ylabel("TPR"); ax.legend(); ax.grid(alpha=grid_alpha)
            st.pyplot(fig, use_container_width=True)

        st.markdown("**Resumen de m√©tricas (test):**")
        st.dataframe(pd.DataFrame(rows).round(3), use_container_width=True)

# ============================================================
# TAB 7 - Insights & Propuestas
# ============================================================
with tabs[6]:
    st.subheader("üìä Insights y Recomendaciones Estrat√©gicas")
    st.caption("Comparaci√≥n de medias entre ciudades usando variables reconstruidas desde los datasets originales (sin intersecci√≥n). Incluye % de Superhosts (T).")

    import re, unicodedata

    # ---- Utilidades robustas ----
    def _norm_txt(s: str) -> str:
        s = str(s)
        s = unicodedata.normalize("NFKD", s)
        s = "".join(ch for ch in s if not unicodedata.combining(ch))
        return s.strip().lower()

    def is_mex(name: str) -> bool:
        n = _norm_txt(name)
        return bool(
            re.search(r"^mex", n) or
            re.search(r"ciudad\s*de\s*mex", n) or
            re.search(r"\bcdmx\b", n) or
            re.fullmatch(r"mx", n)
        )

    def find_col(df_local, aliases):
        cols = list(df_local.columns)
        low = [c.strip().lower() for c in cols]
        # exacta
        for a in aliases:
            a2 = a.strip().lower()
            if a2 in low:
                return cols[low.index(a2)]
        # inclusi√≥n parcial
        for a in aliases:
            a2 = a.strip().lower()
            for i, c in enumerate(low):
                if a2 in c:
                    return cols[i]
        return None

    def clean_numeric_like(series):
        s = series.astype(str).str.replace(r"[^\d,.\-]", "", regex=True)
        mask_coma_decimal = ~s.str.contains(r"\.") & s.str.contains(",")
        s.loc[mask_coma_decimal] = s.loc[mask_coma_decimal].str.replace(",", ".", regex=False)
        s = s.str.replace(",", "", regex=False)
        return pd.to_numeric(s, errors="coerce")

    # ---- Aliases por variable ----
    alias = {
        "host_is_superhost": ["host_is_superhost", "is_superhost", "superhost"],
        "host_response_time": ["host_response_time", "response_time", "tiempo_respuesta"],
        "calculated_host_listings_count": ["calculated_host_listings_count", "host_total_listings_count"],
        "price": ["price", "precio"],
        "number_of_reviews": ["number_of_reviews", "reviews"],
        "availability_365": ["availability_365"],
    }

    # ---- Reconstrucci√≥n desde dfs_raw ----
    recs = []       # filas unificadas
    mappings = []   # mapeo por dataset

    for d in dfs_raw:
        if d is None or d.empty or "city" not in d.columns:
            continue
        df_local = d.copy()
        df_local.columns = [str(c).strip() for c in df_local.columns]

        tmp = pd.DataFrame()
        tmp["city"] = df_local["city"].astype(str)

        # host_is_superhost (con regla para M√©xico)
        col_hsh = find_col(df_local, alias["host_is_superhost"])
        if col_hsh:
            raw = df_local[col_hsh].astype(str).str.strip().str.lower()
            # m√°scara M√©xico
            mex_mask = tmp["city"].map(is_mex).fillna(False)

            # M√©xico: SOLO T/F/1/0 -> map estricta; resto NaN
            map_mex = {"t": 1, "f": 0, "1": 1, "0": 0}
            tmp.loc[mex_mask, "host_is_superhost_num"] = raw[mex_mask].map(map_mex)

            # Otras ciudades: mapeo flexible cl√°sico
            oth = ~mex_mask
            tmp.loc[oth, "host_is_superhost_num"] = np.where(
                raw[oth].isin(["t","true","1","yes","si","s√≠","y","s"]), 1,
                np.where(raw[oth].isin(["f","false","0","no","n"]), 0, np.nan)
            )

            # Tambi√©n versi√≥n F/T para diagn√≥stico y visual
            tmp["host_is_superhost_str"] = tmp["host_is_superhost_num"].map({1.0: "T", 0.0: "F"})
        else:
            tmp["host_is_superhost_num"] = np.nan
            tmp["host_is_superhost_str"] = np.nan

        # host_response_time -> ordinal 4..1
        col_rt = find_col(df_local, alias["host_response_time"])
        if col_rt:
            m = {
                "within an hour": 4, "within a few hours": 3,
                "within a day": 2, "a few days or more": 1
            }
            tmp["host_response_time"] = (
                df_local[col_rt].astype(str).str.strip().str.lower().map(m)
            )
        else:
            tmp["host_response_time"] = np.nan

        # calculated_host_listings_count
        col_chl = find_col(df_local, alias["calculated_host_listings_count"])
        if col_chl:
            tmp["calculated_host_listings_count"] = pd.to_numeric(df_local[col_chl], errors="coerce")
        else:
            tmp["calculated_host_listings_count"] = np.nan

        # price (limpieza general; NaN se ignoran en medias; caso Valencia ya cubierto)
        col_price = find_col(df_local, alias["price"])
        if col_price:
            tmp["price"] = clean_numeric_like(df_local[col_price])
        else:
            tmp["price"] = np.nan

        # number_of_reviews
        col_nor = find_col(df_local, alias["number_of_reviews"])
        if col_nor:
            tmp["number_of_reviews"] = pd.to_numeric(df_local[col_nor], errors="coerce")
        else:
            tmp["number_of_reviews"] = np.nan

        # availability_365
        col_av = find_col(df_local, alias["availability_365"])
        if col_av:
            tmp["availability_365"] = pd.to_numeric(df_local[col_av], errors="coerce")
        else:
            tmp["availability_365"] = np.nan

        recs.append(tmp)

        # mapeo informativo
        maps = {
            "Ciudad(es) en DF": ", ".join(sorted(tmp["city"].astype(str).unique())),
            "host_is_superhost <-": col_hsh if col_hsh else "‚ùå",
            "host_response_time <-": col_rt if col_rt else "‚ùå",
            "calculated_host_listings_count <-": col_chl if col_chl else "‚ùå",
            "price <-": col_price if col_price else "‚ùå",
            "number_of_reviews <-": col_nor if col_nor else "‚ùå",
            "availability_365 <-": col_av if col_av else "‚ùå",
        }
        mappings.append(maps)

    if not recs:
        st.warning("No hay datos disponibles para generar insights.")
        st.stop()

    df_ins = pd.concat(recs, ignore_index=True)
    # Filtra por las ciudades seleccionadas
    df_ins = df_ins[df_ins["city"].isin(sel_cities)].copy()

    # ---- Diagn√≥stico de mapeos ----
    with st.expander("üîé Mapeo de columnas utilizadas por dataset"):
        st.dataframe(pd.DataFrame(mappings), use_container_width=True)

    # ---- Tabla de diagn√≥stico Superhosts F/T por ciudad ----
    with st.expander("üîç Distribuci√≥n F/T por ciudad (host_is_superhost)"):
        if "host_is_superhost_str" in df_ins.columns:
            diag = (
                df_ins.groupby(["city", "host_is_superhost_str"])
                .size()
                .unstack(fill_value=0)
                .rename(columns={"F": "F (No)", "T": "T (S√≠)"})
            )
            st.dataframe(diag, use_container_width=True)

    # ---- C√°lculo de medias por ciudad ----
    metrics = ["host_is_superhost_num", "host_response_time",
               "calculated_host_listings_count", "price",
               "number_of_reviews", "availability_365"]

    summary = {}
    for v in metrics:
        if v in df_ins.columns:
            df_ins[v] = pd.to_numeric(df_ins[v], errors="coerce")
            mean_v = df_ins.groupby("city")[v].mean().round(2)
            if mean_v.notna().any():
                summary[v] = mean_v

    if not summary:
        st.warning("No fue posible calcular medias (todas las series vac√≠as).")
        st.stop()

    df_summary = pd.DataFrame(summary).sort_index()

    # Porcentaje de Superhosts (T) por ciudad
    if "host_is_superhost_num" in df_summary.columns:
        df_summary["%_superhosts"] = (df_summary["host_is_superhost_num"] * 100).round(1)

    st.markdown("### üìà Comparativo de medias entre ciudades")
    st.dataframe(df_summary, use_container_width=True)

    # ---- Gr√°ficas ----
    # Hosts
    st.markdown("#### üßë‚Äçüíº M√©tricas de Hosts")
    for v, title in [
        ("host_is_superhost_num", "% Superhosts (promedio 0/1)"),
        ("host_response_time", "Rapidez de respuesta (4=mejor)"),
        ("calculated_host_listings_count", "Propiedades por host (promedio)"),
    ]:
        if v in df_summary.columns:
            fig = px.bar(df_summary.reset_index(), x="city", y=v, color="city",
                         title=f"{title} ‚Äî promedio por ciudad")
            st.plotly_chart(fig, use_container_width=True)

    # Hospedaje
    st.markdown("#### üè† M√©tricas de Hospedaje")
    for v, title in [
        ("price", "Precio promedio"),
        ("number_of_reviews", "Rese√±as promedio"),
        ("availability_365", "Disponibilidad promedio (d√≠as)"),
    ]:
        if v in df_summary.columns:
            fig = px.bar(df_summary.reset_index(), x="city", y=v, color="city",
                         title=f"{title} ‚Äî promedio por ciudad")
            st.plotly_chart(fig, use_container_width=True)

    # ---- Insights ----
    st.markdown("### üí° Principales Insights")
    insights = []

    # % de Superhosts por ciudad (texto expl√≠cito)
    if "%_superhosts" in df_summary.columns:
        for city, pct in df_summary["%_superhosts"].items():
            insights.append(f"En **{city}**, el **{pct:.1f}%** de los anfitriones son Superhosts (T).")

    def top_city(col, desc, maximize=True):
        if col in df_summary.columns and df_summary[col].notna().any():
            idx = df_summary[col].idxmax() if maximize else df_summary[col].idxmin()
            val = df_summary.loc[idx, col]
            return f"**{idx}** {'lidera' if maximize else 'presenta el menor valor'} en **{desc}** ({val:.2f})."
        return None

    t = top_city("host_response_time", "rapidez de respuesta de anfitriones")
    if t: insights.append(t)
    t = top_city("calculated_host_listings_count", "promedio de propiedades por host")
    if t: insights.append(t)
    t = top_city("price", "precio promedio")
    if t: insights.append(t)
    t = top_city("number_of_reviews", "rese√±as promedio por alojamiento")
    if t: insights.append(t)
    t = top_city("availability_365", "disponibilidad promedio anual")
    if t: insights.append(t)

    if insights:
        for ins in insights:
            st.markdown(f"‚úÖ {ins}")
    else:
        st.info("No se generaron insights autom√°ticos (verifica variables disponibles).")

    # ---- Recomendaciones ----
    st.markdown("### üß≠ Recomendaciones Estrat√©gicas")
    recs = []
    if "%_superhosts" in df_summary.columns:
        minc = df_summary["%_superhosts"].idxmin()
        recs.append(f"- **{minc}**: impulsar la ruta a Superhost (mentor√≠a + checklist de amenities) para elevar el % de superhosts.")
    if "host_response_time" in df_summary.columns:
        minr = df_summary["host_response_time"].idxmin()
        recs.append(f"- **{minr}**: activar respuestas autom√°ticas y recordatorios para mejorar tiempos de respuesta de hosts.")
    if "availability_365" in df_summary.columns:
        maxa = df_summary["availability_365"].idxmax()
        recs.append(f"- **{maxa}**: aplicar pricing din√°mico y promos de temporada baja para reducir vacancia anual.")
    if "price" in df_summary.columns:
        maxp = df_summary["price"].idxmax()
        recs.append(f"- **{maxp}**: consolidar oferta premium y low-budget (bundles: late checkout, limpieza) como estrategia comercial.")
    if "number_of_reviews" in df_summary.columns:
        minrv = df_summary["number_of_reviews"].idxmin()
        recs.append(f"- **{minrv}**: campa√±as de visibilidad + recordatorios post-estancia para aumentar rese√±as.")
    if "calculated_host_listings_count" in df_summary.columns:
        maxh = df_summary["calculated_host_listings_count"].idxmax()
        recs.append(f"- **{maxh}**: estrategia de diversificaci√≥n de alojamientos.")

    for r in recs:
        st.markdown(r)

    st.success("‚úÖ host_is_superhost de M√âXICO tomado correctamente (solo T/F/1/0) y % de T por ciudad calculado.")

Overwriting EVIDENCIA_2.py
